In [1]:
# import dependencies
#Load and inspect images using the Python Imaging Library (PIL)
from PIL import Image
import os
import numpy as np
from PIL import Image
import cv2
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import requests
import pickle
import pandas as pd

In [2]:
path = 'augmented_xray_images.pkl'
with open(path, 'rb') as file:
    imgs = pickle.load(file)

In [3]:
# list values contained in imgs
imgs

[array([[[0.92156863],
         [0.92156863],
         [0.92156863],
         ...,
         [0.9254902 ],
         [0.9254902 ],
         [0.9254902 ]],
 
        [[0.9216688 ],
         [0.92166835],
         [0.9217119 ],
         ...,
         [0.9227088 ],
         [0.9247248 ],
         [0.9254902 ]],
 
        [[0.9254558 ],
         [0.9235696 ],
         [0.92320526],
         ...,
         [0.92428255],
         [0.92534596],
         [0.9254902 ]],
 
        ...,
 
        [[0.9215311 ],
         [0.9197073 ],
         [0.9213665 ],
         ...,
         [0.92156863],
         [0.92156863],
         [0.92156863]],
 
        [[0.9214141 ],
         [0.91965353],
         [0.9213155 ],
         ...,
         [0.92156863],
         [0.92156863],
         [0.92156863]],
 
        [[0.91981345],
         [0.9198711 ],
         [0.9197675 ],
         ...,
         [0.92156863],
         [0.92156863],
         [0.92156863]]], dtype=float32),
 array([[[0.92156863],
         [0.92156

In [8]:
# read y labels for images from csv file
y_df = pd.read_csv("y_augmented_labels.csv", header=None,names=['y_labels'])
# display 1st 5 values
y_df.head()

,y_labels
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign


In [10]:
# replace labels with true if image is not benign but dangerous
# can one hot encode to get type
y_df['y'] = y_df['y_labels']!='Benign'
# print y df
y_df

,y_labels,y
0,Benign,False
1,Benign,False
2,Benign,False
3,Benign,False
4,Benign,False
...,...,...
19010,Throwing Star,True
19011,Throwing Star,True
19012,Throwing Star,True
19013,Throwing Star,True


In [11]:
#print size of images and y label to make sure they are the same length
display(len(imgs), y_df.shape)

19015

(19015, 2)

In [13]:
# define independent and dependent
X = imgs
y = y_df['y']
# Label encode the y data
#y_encoder = LabelEncoder().fit(y)
#y = y_encoder.transform(y)

# Convert values to numpy arrays
X = np.array(X)

# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# below isn't tested
# Define a CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)